In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-stock-market/ind_nifty500list_with_insdustry_name.csv
/kaggle/input/indian-stock-market/NIFTY-500-Perfromance 05-Jan-2025.csv
/kaggle/input/indian-stock-market/Nifty500_data.csv


# Strategy
* Buy the stocks when stock price is below 50 days and 200 days moving average.
* Sell the stock when we booked a profit of 30%.
* In case of loss check the industry trend and if its making loss don't sell. Else sell it if it is making a loss of 15%


### Conclusion
* I tried to optimise the profit and loss with Optuna but it also gave very poor return
* This experiment shows us to get the profit more consistent we need to include other parameters too like fundamental analysis or may be stratagies this approach with other info or may be use some ML concept along with this.

In [2]:
df_info=pd.read_csv('/kaggle/input/indian-stock-market/NIFTY-500-Perfromance 05-Jan-2025.csv')

# Clean column names
# Remove '\n' and blank spaces from column names
df_info.columns = df_info.columns.str.replace(r'\n|\s', '', regex=True)

# industry info df
ind_info_df=pd.read_csv('/kaggle/input/indian-stock-market/ind_nifty500list_with_insdustry_name.csv')

display(ind_info_df.head(2))
print(ind_info_df.Series.unique())
df_info.head(2)

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017


['EQ' 'BE']


,SYMBOL,OPEN,HIGH,LOW,PREV.CLOSE,LTP,INDICATIVECLOSE,CHNG,%CHNG,VOLUME(shares),VALUE(₹Crores),52WH,52WL,30D%CHNG,365D%CHNG03-Jan-2024
0,NIFTY 500,"22,850.10","22,867.85","22,681.30","22,819.75","22,708.15",-,-111.60,-0.49,"2,20,10,40,539","84,132.18","24,573.40","19,080.50",-0.87,17.83
1,ITI,385.50,457.10,379.05,380.95,457.10,-,76.15,19.99,"8,92,46,558","3,794.94",457.10,210.00,32.53,22.18


### What does EQ and BE means?
In the context of the Indian stock market, EQ and BE refer to the Series of stocks traded on stock exchanges like NSE (National Stock Exchange):

1. EQ (Equity) Series:

* EQ represents the normal market segment where equity shares are traded in a rolling settlement system (T+1 or T+2 days).
* This is the most common series for stocks and allows normal trading with intraday and delivery-based transactions.

2. BE (Book Entry) Series:

* BE refers to the trade-for-trade segment.
* Stocks in the BE series are settled on a delivery basis only. This means that intraday trading is not allowed; every buy/sell transaction * has to result in the delivery of shares.
* Typically, stocks in this series are placed here due to regulatory or compliance reasons, such as surveillance measures or illiquidity.

In [3]:
df=pd.read_csv('/kaggle/input/indian-stock-market/Nifty500_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

### removing '.NS' from Stock Ticker
df['Stock Ticker']= df['Stock Ticker'].str.replace('.NS', '', regex=False)

# Convert the column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Format the datetime column to 'year-month-date'
df['Year']= df['Date'].dt.year
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015957 entries, 0 to 1015956
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Date          1015957 non-null  object 
 1   Open          1015956 non-null  float64
 2   High          1015956 non-null  float64
 3   Low           1015956 non-null  float64
 4   Close         1015956 non-null  float64
 5   Volume        1015957 non-null  int64  
 6   Dividends     1015957 non-null  float64
 7   Stock Splits  1015957 non-null  float64
 8   Stock Ticker  1015957 non-null  object 
 9   Year          1015957 non-null  int32  
dtypes: float64(6), int32(1), int64(1), object(2)
memory usage: 73.6+ MB


In [4]:
print(ind_info_df['Symbol'].nunique())
print(df['Stock Ticker'].nunique())

### Checking the symbol Difference (Elements in One Set but Not the Other)
set(ind_info_df['Symbol']) ^ set(df['Stock Ticker'])

501
500


{'DUMMYITC'}

'DUMMYITC' seems some dummy company name in ind_info_df
```python
print(ind_info_df[ind_info_df['Symbol']=='DUMMYITC'].to_markdown(index=False, numalign="left", stralign="left"))
```
| Company Name   | Industry          | Symbol   | Series   | ISIN Code    |
|:---------------|:------------------|:---------|:---------|:-------------|
| Dummy ITC Ltd. | Consumer Services | DUMMYITC | EQ       | DUM154A01025 |

In [5]:
df.head(2)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Stock Ticker,Year
0,2015-01-01,28.350000,29.35,28.350000,28.799999,147964,0.0,0.0,ITI,2015
1,2015-01-02,28.950001,30.25,28.950001,29.250000,662242,0.0,0.0,ITI,2015


In [6]:
%%time
# Sort the data by stock ticker and date
data = df.sort_values(by=['Stock Ticker', 'Date'])

# Calculate 50-day and 200-day moving averages
data['50_MA'] = data.groupby('Stock Ticker')['Close'].transform(lambda x: x.rolling(window=50).mean())
data['200_MA'] = data.groupby('Stock Ticker')['Close'].transform(lambda x: x.rolling(window=200).mean())



CPU times: user 787 ms, sys: 76 ms, total: 863 ms
Wall time: 866 ms


In [7]:
%%time
# Add a Buy Signal column (initialization)
data['Buy_Signal'] = 0
data['Buy_Price'] = None

# Trading strategy
def trading_strategy(df):
    buy_price = None
    buy_triggered = False  # Track if a buy signal has been triggered in a dip cycle

    for index, row in df.iterrows():
        # Check for the first buy signal (price crosses below both 50MA and 200MA)
        if row['Close'] < row['50_MA'] and row['Close'] < row['200_MA'] and not buy_triggered:
            df.at[index, 'Buy_Signal'] = 1
            buy_price = row['Close']  # Record the buy price
            df.at[index, 'Buy_Price'] = buy_price
            buy_triggered = True  # Mark the dip cycle as active

        # Check for the second buy signal (price rises above a threshold, e.g., 5% above the last buy price)
        elif buy_triggered and buy_price is not None:
            threshold_price = buy_price * 1.05  # 5% above the previous buy price
            if row['Close'] > threshold_price:
                df.at[index, 'Buy_Signal'] = 1
                buy_price = row['Close']  # Update the buy price
                df.at[index, 'Buy_Price'] = buy_price
                buy_triggered = False  # Reset the dip cycle

    return df

# Apply the strategy group-wise for each stock ticker
data = data.groupby('Stock Ticker', group_keys=False).apply(trading_strategy)

data.sample(2)

CPU times: user 56.7 s, sys: 172 ms, total: 56.9 s
Wall time: 56.9 s


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Stock Ticker,Year,50_MA,200_MA,Buy_Signal,Buy_Price
601433,2020-05-18,153.216269,155.868086,151.399275,152.528763,656135,0.0,0.0,GRANULES,2020,153.499149,127.455005,0,None
942432,2017-09-20,131.924451,133.350172,130.988814,131.523468,1065653,0.0,0.0,JMFINANCIL,2017,116.410726,91.256455,0,None


In [8]:
%%time
# Adding Sell trigger
# columns to be added 

data['Sell_Signal'] = 0
data['Profit_Loss_Flag'] = None
data['Profit_Loss_Amount'] = None

def sell_logic(df):
    buy_price = None

    for index, row in df.iterrows():
        # Track the most recent buy price
        if row['Buy_Signal'] == 1:
            buy_price = row['Buy_Price']

        # SELL SIGNAL
        if buy_price is not None:
            # Profit condition: Sell if price > 30% of Buy_Price
            if row['Close'] >= buy_price * 1.30:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Profit'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

            # Loss condition: Sell if price < 15% of Buy_Price
            elif row['Close'] <= buy_price * 0.85:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Loss'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

    return df

# Apply sell logic for each stock ticker
data = data.groupby('Stock Ticker', group_keys=False).apply(sell_logic)

# Display results
data.sample(3)

CPU times: user 51.5 s, sys: 118 ms, total: 51.6 s
Wall time: 51.6 s


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Stock Ticker,Year,50_MA,200_MA,Buy_Signal,Buy_Price,Sell_Signal,Profit_Loss_Flag,Profit_Loss_Amount
992405,2022-08-05,5123.819329,5171.690156,4922.406323,5043.906738,91652,0.0,0.0,BAJAJHLDNG,2022,4656.915635,4858.920420,0,None,0,None,None
337217,2022-06-06,922.937332,948.968387,911.184464,937.895447,960521,0.0,0.0,COROMANDEL,2022,851.008054,783.227168,0,None,0,None,None
122208,2018-02-26,74.430489,80.906906,73.657183,76.315414,55707787,0.0,0.0,IDBI,2018,60.418222,57.537907,0,None,0,None,None


In [9]:
%%time
# Hypothesis generation
insights_df= data[['Buy_Signal','Year','Buy_Price','Sell_Signal','Profit_Loss_Amount']].groupby('Year').agg(
    Total_Buy_signal_count=('Buy_Signal', 'sum'),
    Total_Sell_signal_count=('Sell_Signal', 'sum'),
    Total_buy_amount=('Buy_Price', 'sum'),
    Total_profit_loss=('Profit_Loss_Amount', 'sum'),
    Avg_buy_price=('Buy_Price', 'mean')
    ).reset_index()

insights_df['Total_returns Percentage']= 100* insights_df['Total_profit_loss']/insights_df['Total_buy_amount']
insights_df

CPU times: user 166 ms, sys: 2.99 ms, total: 169 ms
Wall time: 171 ms


,Year,Total_Buy_signal_count,Total_Sell_signal_count,Total_buy_amount,Total_profit_loss,Avg_buy_price,Total_returns Percentage
0,2015,418,27,245263.144255,-3855.155035,586.753934,-1.571844
1,2016,1172,322,772474.583846,6890.07484,659.108007,0.891948
2,2017,868,214,640636.718629,32834.713117,738.060736,5.125325
3,2018,1237,326,1206550.904161,-17657.170045,975.384724,-1.463442
4,2019,710,138,796584.02183,5941.21228,1121.949327,0.745836
5,2020,734,286,1240010.061502,3480.108867,1689.387005,0.280652
6,2021,753,219,1435023.362984,63627.318541,1905.741518,4.433887
7,2022,1524,327,2230537.741455,-46664.241205,1463.607442,-2.092062
8,2023,831,259,1269819.603307,70392.25154,1528.062098,5.543484
9,2024,1549,282,3268556.312904,68741.870424,2110.107368,2.103126


### Optimise the profit and loss parameters

In [10]:
# Sort the data by stock ticker and date
data1 = df.sort_values(by=['Stock Ticker', 'Date'])

# Calculate 50-day and 200-day moving averages
data1['50_MA'] = data.groupby('Stock Ticker')['Close'].transform(lambda x: x.rolling(window=50).mean())
data1['200_MA'] = data.groupby('Stock Ticker')['Close'].transform(lambda x: x.rolling(window=200).mean())
data1 =data1[['Stock Ticker','Date','Year','Close','50_MA','200_MA']]
dff = data1.copy()

In [11]:
dff.head()

,Stock Ticker,Date,Year,Close,50_MA,200_MA
481830,360ONE,2019-09-19,2019,262.504517,NaN,NaN
481831,360ONE,2019-09-20,2019,275.624603,NaN,NaN
481832,360ONE,2019-09-23,2019,289.405823,NaN,NaN
481833,360ONE,2019-09-24,2019,303.868927,NaN,NaN
481834,360ONE,2019-09-25,2019,290.283936,NaN,NaN


In [12]:
%%time
# Add a Buy Signal column (initialization)
dff['Buy_Signal'] = 0
dff['Buy_Price'] = None
dff['Sell_Signal'] = 0
dff['Profit_Loss_Flag'] = None
dff['Profit_Loss_Amount'] = None



# Trading strategy
def buy_logic(df):
    buy_price = None
    buy_triggered = False  # Track if a buy signal has been triggered in a dip cycle

    for index, row in df.iterrows():
        # Check for the first buy signal (price crosses below both 50MA and 200MA)
        if row['Close'] < row['50_MA'] and row['Close'] < row['200_MA'] and not buy_triggered:
            df.at[index, 'Buy_Signal'] = 1
            buy_price = row['Close']  # Record the buy price
            df.at[index, 'Buy_Price'] = buy_price
            buy_triggered = True  # Mark the dip cycle as active

        # Check for the second buy signal (price rises above a threshold, e.g., 5% above the last buy price)
        elif buy_triggered and buy_price is not None:
            threshold_price = buy_price * 1.05  # 5% above the previous buy price
            if row['Close'] > threshold_price:
                df.at[index, 'Buy_Signal'] = 1
                buy_price = row['Close']  # Update the buy price
                df.at[index, 'Buy_Price'] = buy_price
                buy_triggered = False  # Reset the dip cycle

    return df

def sell_logic(df):
    buy_price = None

    for index, row in df.iterrows():
        # Track the most recent buy price
        if row['Buy_Signal'] == 1:
            buy_price = row['Buy_Price']

        # SELL SIGNAL
        if buy_price is not None:
            # Profit condition: Sell if price > 30% of Buy_Price
            if row['Close'] >= buy_price * 1.30:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Profit'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

            # Loss condition: Sell if price < 30% of Buy_Price
            elif row['Close'] <= buy_price * 0.70:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Loss'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

    return df

# Apply the strategy group-wise for each stock ticker
dff = dff.groupby('Stock Ticker', group_keys=False).apply(buy_logic)
dff = dff.groupby('Stock Ticker', group_keys=False).apply(sell_logic)
dff.sample(3)

CPU times: user 1min 51s, sys: 75.7 ms, total: 1min 51s
Wall time: 1min 51s


,Stock Ticker,Date,Year,Close,50_MA,200_MA,Buy_Signal,Buy_Price,Sell_Signal,Profit_Loss_Flag,Profit_Loss_Amount
141042,GAEL,2021-11-23,2021,83.404770,85.642129,80.412498,0,None,0,None,None
393447,IOC,2023-04-21,2023,69.287766,69.434750,65.529657,0,None,0,None,None
742,ITI,2018-01-03,2018,140.800003,142.583000,109.746250,0,None,0,None,None


In [13]:
insights_dff= dff[['Buy_Signal','Year','Buy_Price','Sell_Signal','Profit_Loss_Amount']].groupby('Year').agg(
    Total_Buy_signal_count=('Buy_Signal', 'sum'),
    Total_Sell_signal_count=('Sell_Signal', 'sum'),
    Total_buy_amount=('Buy_Price', 'sum'),
    Total_profit_loss=('Profit_Loss_Amount', 'sum'),
    Avg_buy_price=('Buy_Price', 'mean')
    ).reset_index()

insights_dff['Total_returns Percentage']= 100* insights_dff['Total_profit_loss']/insights_dff['Total_buy_amount']
insights_dff

,Year,Total_Buy_signal_count,Total_Sell_signal_count,Total_buy_amount,Total_profit_loss,Avg_buy_price,Total_returns Percentage
0,2015,418,4,245263.144255,11.237785,586.753934,0.004582
1,2016,1172,186,772474.583846,22662.588204,659.108007,2.933765
2,2017,868,180,640636.718629,34034.099136,738.060736,5.312543
3,2018,1237,192,1206550.904161,3872.158117,975.384724,0.320928
4,2019,710,139,796584.02183,-9456.606363,1121.949327,-1.187145
5,2020,734,234,1240010.061502,12750.630283,1689.387005,1.028268
6,2021,753,179,1435023.362984,70645.391194,1905.741518,4.922944
7,2022,1524,190,2230537.741455,-21205.721697,1463.607442,-0.9507
8,2023,831,229,1269819.603307,74326.380664,1528.062098,5.853302
9,2024,1549,184,3268556.312904,91570.033834,2110.107368,2.801544


In [21]:
%%time
# Add a Buy Signal column (initialization)
dff['Buy_Signal'] = 0
dff['Buy_Price'] = None
dff['Sell_Signal'] = 0
dff['Profit_Loss_Flag'] = None
dff['Profit_Loss_Amount'] = None



# Trading strategy
def buy_logic(df):
    buy_price = None
    buy_triggered = False  # Track if a buy signal has been triggered in a dip cycle

    for index, row in df.iterrows():
        # Check for the first buy signal (price crosses below both 50MA and 200MA)
        if row['Close'] < row['50_MA'] and row['Close'] < row['200_MA'] and not buy_triggered:
            df.at[index, 'Buy_Signal'] = 1
            buy_price = row['Close']  # Record the buy price
            df.at[index, 'Buy_Price'] = buy_price
            buy_triggered = True  # Mark the dip cycle as active

        # Check for the second buy signal (price rises above a threshold, e.g., 5% above the last buy price)
        elif buy_triggered and buy_price is not None:
            threshold_price = buy_price * 1.05  # 5% above the previous buy price
            if row['Close'] > threshold_price:
                df.at[index, 'Buy_Signal'] = 1
                buy_price = row['Close']  # Update the buy price
                df.at[index, 'Buy_Price'] = buy_price
                buy_triggered = False  # Reset the dip cycle

    return df

def sell_logic(df):
    buy_price = None

    for index, row in df.iterrows():
        # Track the most recent buy price
        if row['Buy_Signal'] == 1:
            buy_price = row['Buy_Price']

        # SELL SIGNAL
        if buy_price is not None:
            # Profit condition: Sell if price > 30% of Buy_Price
            if row['Close'] >= buy_price * 1.30:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Profit'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

            # Loss condition: Sell if price < 50% of Buy_Price
            elif row['Close'] <= buy_price * 0.50:
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Loss'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None  # Reset buy price after selling

    return df

# Apply the strategy group-wise for each stock ticker
dff = dff.groupby('Stock Ticker', group_keys=False).apply(buy_logic)
dff = dff.groupby('Stock Ticker', group_keys=False).apply(sell_logic)
dff.sample(3)

insights_dff= dff[['Buy_Signal','Year','Buy_Price','Sell_Signal','Profit_Loss_Amount']].groupby('Year').agg(
    Total_Buy_signal_count=('Buy_Signal', 'sum'),
    Total_Sell_signal_count=('Sell_Signal', 'sum'),
    Total_buy_amount=('Buy_Price', 'sum'),
    Total_profit_loss=('Profit_Loss_Amount', 'sum'),
    Avg_buy_price=('Buy_Price', 'mean')
    ).reset_index()

insights_dff['Total_returns Percentage']= 100* insights_dff['Total_profit_loss']/insights_dff['Total_buy_amount']
insights_dff

CPU times: user 1min 53s, sys: 136 ms, total: 1min 53s
Wall time: 1min 54s


,Year,Total_Buy_signal_count,Total_Sell_signal_count,Total_buy_amount,Total_profit_loss,Avg_buy_price,Total_returns Percentage
0,2015,418,3,245263.144255,15.985963,586.753934,0.006518
1,2016,1172,133,772474.583846,28511.157026,659.108007,3.690886
2,2017,868,177,640636.718629,32619.459692,738.060736,5.091725
3,2018,1237,99,1206550.904161,9433.792607,975.384724,0.781881
4,2019,710,119,796584.02183,9987.630163,1121.949327,1.253807
5,2020,734,235,1240010.061502,-3957.276914,1689.387005,-0.319133
6,2021,753,176,1435023.362984,71097.123799,1905.741518,4.954423
7,2022,1524,112,2230537.741455,18045.592328,1463.607442,0.809024
8,2023,831,204,1269819.603307,87084.492332,1528.062098,6.858021
9,2024,1549,167,3268556.312904,95393.114576,2110.107368,2.918509


# Optuna
I will now try to use Optuna to optimise the profit and loss

In [14]:
import optuna

In [15]:
%%time
# Initialize columns
dff['Buy_Signal'] = 0
dff['Buy_Price'] = None
dff['Sell_Signal'] = 0
dff['Profit_Loss_Flag'] = None
dff['Profit_Loss_Amount'] = None

# Buy logic function
def buy_logic(df):
    buy_price = None
    buy_triggered = False

    for index, row in df.iterrows():
        if row['Close'] < row['50_MA'] and row['Close'] < row['200_MA'] and not buy_triggered:
            df.at[index, 'Buy_Signal'] = 1
            buy_price = row['Close']
            df.at[index, 'Buy_Price'] = buy_price
            buy_triggered = True

        elif buy_triggered and buy_price is not None:
            threshold_price = buy_price * 1.05
            if row['Close'] > threshold_price:
                df.at[index, 'Buy_Signal'] = 1
                buy_price = row['Close']
                df.at[index, 'Buy_Price'] = buy_price
                buy_triggered = False

    return df

# Sell logic function
def sell_logic(df, profit_threshold, loss_threshold):
    buy_price = None

    for index, row in df.iterrows():
        if row['Buy_Signal'] == 1:
            buy_price = row['Buy_Price']

        if buy_price is not None:
            if row['Close'] >= buy_price * (1 + profit_threshold):
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Profit'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None

            elif row['Close'] <= buy_price * (1 - loss_threshold):
                df.at[index, 'Sell_Signal'] = 1
                df.at[index, 'Profit_Loss_Flag'] = 'Loss'
                df.at[index, 'Profit_Loss_Amount'] = row['Close'] - buy_price
                buy_price = None

    return df

# Objective function for Optuna
def objective(trial):
    # Define profit and loss thresholds as hyperparameters
    profit_threshold = trial.suggest_float('profit_threshold', 0.05, 0.50, step=0.01)
    loss_threshold = trial.suggest_float('loss_threshold', 0.05, 0.50, step=0.01)

    # Reset data for each trial
    df = dff.copy()
    df = df.groupby('Stock Ticker', group_keys=False).apply(buy_logic)
    df = df.groupby('Stock Ticker', group_keys=False).apply(sell_logic, profit_threshold=profit_threshold, loss_threshold=loss_threshold)

    # Calculate profitability insights
    insights_df = df[['Buy_Signal', 'Year', 'Buy_Price', 'Sell_Signal', 'Profit_Loss_Amount']].groupby('Year').agg(
        Total_Buy_signal_count=('Buy_Signal', 'sum'),
        Total_Sell_signal_count=('Sell_Signal', 'sum'),
        Total_buy_amount=('Buy_Price', 'sum'),
        Total_profit_loss=('Profit_Loss_Amount', 'sum'),
        Avg_buy_price=('Buy_Price', 'mean')
    ).reset_index()

    insights_df['Total_returns_Percentage'] = 100 * insights_df['Total_profit_loss'] / insights_df['Total_buy_amount']

    # Maximize total returns percentage
    total_returns = insights_df['Total_returns_Percentage'].sum()
    return total_returns

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best parameters and results
print(f"Best Profit Threshold: {study.best_params['profit_threshold']}")
print(f"Best Loss Threshold: {study.best_params['loss_threshold']}")
print(f"Best Total Returns: {study.best_value}")

# Apply the best parameters to the data
best_profit_threshold = study.best_params['profit_threshold']
best_loss_threshold = study.best_params['loss_threshold']

dff = dff.groupby('Stock Ticker', group_keys=False).apply(buy_logic)
dff = dff.groupby('Stock Ticker', group_keys=False).apply(
    sell_logic, 
    profit_threshold=best_profit_threshold, 
    loss_threshold=best_loss_threshold
)

# Generate insights
insights_dff = dff[['Buy_Signal', 'Year', 'Buy_Price', 'Sell_Signal', 'Profit_Loss_Amount']].groupby('Year').agg(
    Total_Buy_signal_count=('Buy_Signal', 'sum'),
    Total_Sell_signal_count=('Sell_Signal', 'sum'),
    Total_buy_amount=('Buy_Price', 'sum'),
    Total_profit_loss=('Profit_Loss_Amount', 'sum'),
    Avg_buy_price=('Buy_Price', 'mean')
).reset_index()

insights_dff['Total_returns_Percentage'] = 100 * insights_dff['Total_profit_loss'] / insights_dff['Total_buy_amount']
print(insights_dff)

[I 2025-01-06 11:07:28,144] A new study created in memory with name: no-name-4de5f82b-539f-4d96-bb91-f63b26ab55f6
[I 2025-01-06 11:09:20,739] Trial 0 finished with value: 20.972964731326215 and parameters: {'profit_threshold': 0.19, 'loss_threshold': 0.39}. Best is trial 0 with value: 20.972964731326215.
[I 2025-01-06 11:11:12,301] Trial 1 finished with value: 14.332364042388328 and parameters: {'profit_threshold': 0.28, 'loss_threshold': 0.12000000000000001}. Best is trial 0 with value: 20.972964731326215.
[I 2025-01-06 11:13:04,654] Trial 2 finished with value: 17.6697075601515 and parameters: {'profit_threshold': 0.3, 'loss_threshold': 0.26}. Best is trial 0 with value: 20.972964731326215.
[I 2025-01-06 11:14:56,044] Trial 3 finished with value: 5.7248255667944346 and parameters: {'profit_threshold': 0.09, 'loss_threshold': 0.19}. Best is trial 0 with value: 20.972964731326215.
[I 2025-01-06 11:16:48,162] Trial 4 finished with value: 15.62917710218376 and parameters: {'profit_thresh

Best Profit Threshold: 0.46
Best Loss Threshold: 0.47
Best Total Returns: 36.39803209858283
    Year  Total_Buy_signal_count  Total_Sell_signal_count Total_buy_amount  \
0   2015                     418                        1    245263.144255   
1   2016                    1172                       90    772474.583846   
2   2017                     868                      143    640636.718629   
3   2018                    1237                       84   1206550.904161   
4   2019                     710                       94     796584.02183   
5   2020                     734                      205   1240010.061502   
6   2021                     753                      151   1435023.362984   
7   2022                    1524                       77   2230537.741455   
8   2023                     831                      158   1269819.603307   
9   2024                    1549                      146   3268556.312904   
10  2025                      16                  

In [16]:
insights_dff

,Year,Total_Buy_signal_count,Total_Sell_signal_count,Total_buy_amount,Total_profit_loss,Avg_buy_price,Total_returns_Percentage
0,2015,418,1,245263.144255,9.128681,586.753934,0.003722
1,2016,1172,90,772474.583846,10768.206287,659.108007,1.393988
2,2017,868,143,640636.718629,49083.762067,738.060736,7.661715
3,2018,1237,84,1206550.904161,11264.593405,975.384724,0.933619
4,2019,710,94,796584.02183,769.286328,1121.949327,0.096573
5,2020,734,205,1240010.061502,4279.88195,1689.387005,0.345149
6,2021,753,151,1435023.362984,76152.115563,1905.741518,5.306681
7,2022,1524,77,2230537.741455,9888.999331,1463.607442,0.443346
8,2023,831,158,1269819.603307,88984.641376,1528.062098,7.00766
9,2024,1549,146,3268556.312904,98047.548759,2110.107368,2.99972


In [18]:
dff.to_csv('Final_data_SMA.csv', index=False)